In [ ]:
# add module to path so python can import it. This is the directory with 
# the __init__.py file in it to let python know this is a module.
import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/spicy_snow')

from download.sentinel1 import combine_s1_images

In [ ]:
import numpy as np
import xarray as xr

das = {}
granules = ['S1B_IW_GRDH_1SDV_20200201T013528_20200201T013553_020069_025FB3_6D5E',
'S1B_IW_GRDH_1SDV_20200130T134920_20200130T134948_020047_025EF4_9218',
'S1B_IW_GRDH_1SDV_20200127T012726_20200127T012751_019996_025D37_D0F0',
'S1A_IW_GRDH_1SDV_20200126T013605_20200126T013634_030965_038E37_760E',
'S1A_IW_GRDH_1SDV_20200124T135002_20200124T135027_030943_038D6F_FED5']
for i in range(5):
    backscatter = np.random.randn(10, 10, 3)
    x = np.linspace(0, 9, 10)
    y = np.linspace(10, 19, 10)
    lon, lat = np.meshgrid(x, y)

    da = xr.DataArray(data = backscatter,
        dims = ["x","y","band"],
        coords = dict(
            lon=(["x", "y"], lon),
            lat=(["x", "y"], lat)))
    das[granules[i]] = da

In [ ]:
ds = combine_s1_images(das)

In [ ]:
ds.flight_dir

In [ ]:
# standard imports
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt

# load up the saved dataset
import pickle
fp = '/Users/zachkeskinen/Documents/spicy-snow/tests/test_data/2_img_ds'
# fp = '/Users/zachkeskinen/Documents/spicy-snow/data/10_img_dB.pkl'
with open(fp, 'rb') as f:
    ds = pickle.load(f)

# add module to path so python can import it. This is the directory with 
# the __init__.py file in it to let python know this is a module.
import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/spicy_snow')

# import the function to test
from processing.snow_index import calc_delta_VV, calc_delta_cross_ratio

# ds = ds.isel(time = slice(1, 3))

ds = calc_delta_cross_ratio(ds)
ds = calc_delta_VV(ds)

# import the function to test
from processing.snow_index import calc_delta_gamma, clip_delta_gamma_outlier
ds = calc_delta_gamma(ds)

In [ ]:
# import the function to test
from spicy_snow.download.sentinel1 import s1_img_search, hyp3_pipeline, download_hyp3, combine_s1_images
import shapely
import os
# sets dates and areas
dates = ('2019-02-01', '2019-02-25')
area = shapely.geometry.box(-115, 43, -114, 44)

# Set skip = True to skip indented steps
skip = False


os.makedirs('../data' , exist_ok = True)
# get asf_search search results
search_results = s1_img_search(area, dates)
print(f'Found {len(search_results)} results')

In [ ]:
# download s1 images into dataset ['s1'] keyword
jobs = hyp3_pipeline(search_results, job_name = 'improve-downloads', existing_job_name = 'improve-downloads')

In [ ]:
das = download_hyp3(jobs, area, outdir='/Users/zachkeskinen/Documents/spicy-snow/data/tmp4', clean = False)

In [ ]:
ds = combine_s1_images(das)

In [ ]:
ds['s1'].isel(time = 1).sel(band = 'VH').plot()